In [3]:
import construction as cs
import matplotlib.pyplot as plt

### read font
from matplotlib import font_manager

font_dirs = ['Barlow/']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

# set font
plt.rcParams['font.family'] = 'Barlow'

import networkx as nx

import networkx as nx
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

from topological_metrics import *

import os

# example of calculating the kl divergence between two mass functions
from math import log2
from scipy.stats import wasserstein_distance as em
from scipy.stats import ks_2samp

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Different distances topology

In [1]:
def compute_probs(data, n=10): 
    data = np.array(data)
    h, e = np.histogram(data, n)
    p = h/data.shape[0]
    return e, p

def support_intersection(p, q): 
    sup_int = (
        list(
            filter(
                lambda x: (x[0]!=0) & (x[1]!=0), zip(p, q)
            )
        )
    )
    return sup_int

def get_probs(list_of_tuples): 
    p = np.array([p[0] for p in list_of_tuples])
    q = np.array([p[1] for p in list_of_tuples])
    return p, q

def kl_divergence(p, q): 
    return np.sum(p*np.log(p/q))

def js_divergence(p, q):
    m = (1./2.)*(p + q)
    return (1./2.)*kl_divergence(p, m) + (1./2.)*kl_divergence(q, m)

def compute_kl_divergence(train_sample, test_sample, n_bins=10,js=False): 
    """
    Computes the KL Divergence using the support 
    intersection between two different samples
    """
    E = 0.0000000001
    e, p = compute_probs(train_sample, n=n_bins)
    _, q = compute_probs(test_sample, n=e)
    
    p = np.array(p) + E 
    q = np.array(q) + E 
    
    p = p/sum(p)
    q = q/sum(q)
    
    list_of_tuples = support_intersection(p, q)
    p, q = get_probs(list_of_tuples)
    
    if js:
        return js_divergence(p, q)
    else:
        return kl_divergence(p, q)
def comp_stat(ori,competitor,dist,names):
    res = dict()
    c = 0
    for met in competitor:
        tmp = []
        for comp in met:
            if dist == "js":
                val = compute_kl_divergence(ori[c], comp, n_bins=50,js=True)
            elif dist == "kl":
                val = compute_kl_divergence(ori[c], comp, n_bins=50,js=False)
            elif dist == "em":
                val = em(ori[c][0],comp)
            elif dist == "ks":
                val = (ks_2samp(ori[c][0],comp)[0])
                
            tmp.append(val)
            
        res[names[c]] = (np.mean(tmp),np.std(tmp))
        c = c + 1 
        
    return res




def load_comp_metric(file_name,dist="ks"):
    o_in = load_topo_original(file_name)
    e_in = load_topo_distributions("etngen",file_name)
    t_in = load_topo_distributions("taggen",file_name)
    d_in = load_topo_distributions("dymgen",file_name)
    s_in = load_topo_distributions("stmgen",file_name)

    res_e = comp_stat(o_in,e_in,dist= dist,names=names)
    res_d = comp_stat(o_in,d_in,dist= dist,names=names)
    res_s = comp_stat(o_in,s_in,dist= dist,names=names)
    res_t = comp_stat(o_in,t_in,dist= dist,names=names)

    x1 = np.array(list(res_e.values()))
    x2 = np.array(list(res_s.values()))
    x3 = np.array(list(res_t.values()))
    x4 = np.array(list(res_d.values()))
    
    return x1,x2,x3,x4


In [ ]:
def load_topo_distributions(generator,file_name):
    
    den = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/density.npy",allow_pickle=True)
    inter_indiv = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/interacting_indiv.npy",allow_pickle=True)
    new_conv = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/new_con.npy",allow_pickle=True)
    durat = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/dur.npy",allow_pickle=True)
    clust = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/clust.npy",allow_pickle=True)
    #s_met = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/s_metric.npy",allow_pickle=True)
    ass = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/ass.npy",allow_pickle=True)
    #asp = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/asp.npy",allow_pickle=True)
    hclose = np.load("topology_results/topology_results_giulia/"+generator+"/Multiple_run/distributions/"+file_name+"/hclose.npy",allow_pickle=True)
    hbet = np.load("topology_results/topology_results_giulia/"+generator+"/Multiple_run/distributions/"+file_name+"/hbet.npy",allow_pickle=True)
    whbet = np.load("topology_results/topology_results_giulia/"+generator+"/Multiple_run/distributions/"+file_name+"/whbet.npy",allow_pickle=True)
    conncomp = np.load("topology_results/topology_results_giulia/"+generator+"/Multiple_run/distributions/"+file_name+"/conncomp.npy",allow_pickle=True)
    hmod = np.load("topology_results/topology_results_giulia/"+generator+"/Multiple_run/distributions/"+file_name+"/hmod.npy",allow_pickle=True)
    
    hs_met = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/hs_metric.npy",allow_pickle=True)
    hasp = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/hasp.npy",allow_pickle=True)
    #nb_inter = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/nb_interactions.npy",allow_pickle=True)
    #stren = np.load("topology_results/"+generator+"/Multiple_run/distributions/"+file_name+"/streng.npy",allow_pickle=True 

    #return den,inter_indiv,new_conv,durat,clust,ass,hclose,hbet,whbet,conncomp,hmod,hs_met,hasp
    return den,inter_indiv,new_conv,durat,clust,ass,conncomp,hclose,hbet,whbet,hmod,hs_met,hasp

def load_topo_original(file_name):
    den = np.load("topology_results/original_distributions/"+file_name+"/density.npy",allow_pickle=True)
    inter_indiv = np.load("topology_results/original_distributions/"+file_name+"/interacting_indiv.npy",allow_pickle=True)
    new_conv = np.load("topology_results/original_distributions/"+file_name+"/new_con.npy",allow_pickle=True)
    durat = np.load("topology_results/original_distributions/"+file_name+"/dur.npy",allow_pickle=True)
    clust = np.load("topology_results/original_distributions/"+file_name+"/clust.npy",allow_pickle=True)
    ass = np.load("topology_results/original_distributions/"+file_name+"/ass.npy",allow_pickle=True)
    hclose = np.load("topology_results/topology_results_giulia/original_distributions/"+file_name+"/hclose.npy",allow_pickle=True)
    hbet = np.load("topology_results/topology_results_giulia/original_distributions/"+file_name+"/hbet.npy",allow_pickle=True)
    whbet = np.load("topology_results/topology_results_giulia/original_distributions/"+file_name+"/whbet.npy",allow_pickle=True)
    conncomp = np.load("topology_results/topology_results_giulia/original_distributions/"+file_name+"/conncomp.npy",allow_pickle=True)
    hmod = np.load("topology_results/topology_results_giulia/original_distributions/"+file_name+"/hmod.npy",allow_pickle=True)
    
    hs_met = np.load("topology_results/original_distributions/"+file_name+"/hs_metric.npy",allow_pickle=True)
    hasp = np.load("topology_results/original_distributions/"+file_name+"/hasp.npy",allow_pickle=True)
    
    
    
    #s_met = np.load("topology_results/original_distributions/"+file_name+"/s_metric.npy",allow_pickle=True)
    #asp = np.load("topology_results/original_distributions/"+file_name+"/asp.npy",allow_pickle=True)
    #stren = np.load("topology_results/original_distributions/"+file_name+"/streng.npy",allow_pickle=True)
    #nb_inter = np.load("topology_results/original_distributions/"+file_name+"/nb_interactions.npy",allow_pickle=True)
    
    return den,inter_indiv,new_conv,durat,clust,ass,[conncomp],[hclose],[hbet],[whbet],[hmod],[hs_met],[hasp]



In [ ]:
names = ["Density","Interacting  individuals","New conversations","Duration of contacts","Global clustering coefficient",
         "Assortativity","Nb connected components","Hour closeness","Hour betweenness","Weighted hour betweenness",
         "Hour modularity","Hour S-metric","Hour average shortestpath length"]

# etn stm tag dym
#x1w,x2w,x3w,x4w = load_comp_metric("InVS13",dist=dist)
#x1s,x2s,x3s,x4s = load_comp_metric("High_School11",dist=dist)

In [ ]:
file_name  = "High_School11"


x1h,x2h,x3h,x4h = load_comp_metric(file_name,dist="ks")
tmp = []
for i in range(len(x1h)):
    a = x1h[i][0]
    b = x2h[i][0]
    c = x3h[i][0]
    d = x4h[i][0]
    tmp.append(["{:.2f}".format(a),"{:.2f}".format(b),"{:.2f}".format(c),"{:.2f}".format(d)])
    
x1h,x2h,x3h,x4h = load_comp_metric(file_name,dist="js")
cc = 0
for i in range(len(x1h)):
    a = x1h[i][0]
    b = x2h[i][0]
    c = x3h[i][0]
    d = x4h[i][0]
    tmp[cc].extend(["{:.2f}".format(a),"{:.2f}".format(b),"{:.2f}".format(c),"{:.2f}".format(d)])
    cc = cc + 1
    
x1h,x2h,x3h,x4h = load_comp_metric(file_name,dist="kl")
cc = 0
for i in range(len(x1h)):
    a = x1h[i][0]
    b = x2h[i][0]
    c = x3h[i][0]
    d = x4h[i][0]
    tmp[cc].extend(["{:.2f}".format(a),"{:.2f}".format(b),"{:.2f}".format(c),"{:.2f}".format(d)])
    cc = cc + 1
    
x1h,x2h,x3h,x4h = load_comp_metric(file_name,dist="em")
cc = 0
for i in range(len(x1h)):
    a = x1h[i][0]
    b = x2h[i][0]
    c = x3h[i][0]
    d = x4h[i][0]
    tmp[cc].extend(["{:.5f}".format(a),"{:.5f}".format(b),"{:.5f}".format(c),"{:.5f}".format(d)])
    cc = cc + 1

In [ ]:
names2 = ["Density ","Int. ind.","New conv.","Dur.","GCC","Ass.","Con. com.","H. clos.","H betw.","W. h. betw.",
"H. modu."   ,
"H. S-met."  ,
"H. aspl"]

In [ ]:
c = 0
for i in tmp:
    s = names2[c]+"&"
    for j in i:
        s = s + j + " & "
    c = c +1 
    print(s[:-2]+"\\\\")

In [ ]:
for i in names:
    print(i)

In [ ]:
0.08 & 0.07 & 0.05 & 0.38 \\
0.09 & 0.07 & 0.06 & 0.37 \\

# Different distances dynamic

# SIR

In [5]:

def load_r0(file_name,lambds=["la001"],picco=""):
    ori = []
    stb = []
    etn = []
    tag = []
    stm = []
    dym = []
    for lambd in lambds:
        ori.append(np.load("dynamic_results/"+file_name+"/Multiple_run/R0"+picco+"/"+lambd+"/orig.npy"))
        stb.append(np.load("dynamic_results/"+file_name+"/Multiple_run/R0"+picco+"/"+lambd+"/stab.npy"))
        etn.append(np.load("dynamic_results/"+file_name+"/Multiple_run/R0"+picco+"/"+lambd+"/etn.npy"))
        tag.append(np.load("dynamic_results/"+file_name+"/Multiple_run/R0"+picco+"/"+lambd+"/tag.npy"))
        stm.append(np.load("dynamic_results/"+file_name+"/Multiple_run/R0"+picco+"/"+lambd+"/stm.npy"))
        dym.append(np.load("dynamic_results/"+file_name+"/Multiple_run/R0"+picco+"/"+lambd+"/dym.npy"))
        
    #return ori,stb,etn,stm,tag,dym
    #ori,stb,etn,tag,dym stm
    return ori,stb,etn,tag,dym,stm
import math



# example of calculating the kl divergence between two mass functions
from math import log2
from scipy.stats import wasserstein_distance as em
from scipy.stats import ks_2samp



def comp_stat(ori,competitor,dist,names):
    res = dict()
    c = 0
    for met in competitor:
        tmp = []
        for comp in met:
            if dist == "js":
                val = compute_kl_divergence(ori[c], comp, n_bins=50,js=True)
            elif dist == "kl":
                val = compute_kl_divergence(ori[c], comp, n_bins=50,js=False)
            elif dist == "em":
                val = em(ori[c],comp)
            elif dist == "ks":
                val = (ks_2samp(ori[c],comp)[0])
                
            tmp.append(val)
            
        res[names[c]] = (np.mean(tmp),np.std(tmp))
        c = c + 1 
        
    return res
def comp_stat_stb(ori,competitor,dist,names):
    
    res = dict()
    c = 0
    for comp in competitor:
        tmp = []
        if dist == "js":
            val = compute_kl_divergence(ori[c], comp, n_bins=50,js=True)
        elif dist == "kl":
            val = compute_kl_divergence(ori[c], comp, n_bins=50,js=False)
        elif dist == "em":
            val = em(ori[c],comp)
        elif dist == "ks":
            val = (ks_2samp(ori[c],comp)[0])

        tmp.append(val)
            
        res[names[c]] = (np.mean(tmp),np.std(tmp))
        c = c + 1 
        
    return res

In [54]:
file_name = "High_School11"
lambs_all = ["la025","la015","la001"]
picchi = ["","_meta","_2picco"]
dist ="ks"


for picco in picchi:
    for lambs in lambs_all:
        k = lambs
        lambs = [lambs]
        ori,stb,etn,tag,dym,stm = load_r0(file_name,lambds=lambs,picco=picco)
        
        s = picco
        s += k+","
        dist = "js"
        r_st = comp_stat_stb(ori,stb,dist,lambs)
        r_e = comp_stat(ori,etn,dist,lambs)
        r_t = comp_stat(ori,tag,dist,lambs)
        r_d = comp_stat(ori,dym,dist,lambs)
        r_s = comp_stat(ori,stm,dist,lambs)

        
        s += str(r_e[k][0])+","
        s += str(r_t[k][0])+","
        s += str(r_s[k][0])+","
        s += str(r_d[k][0])+"," 


        dist = "kl"
        r_st = comp_stat_stb(ori,stb,dist,lambs)
        r_e = comp_stat(ori,etn,dist,lambs)
        r_t = comp_stat(ori,tag,dist,lambs)
        r_d = comp_stat(ori,dym,dist,lambs)
        r_s = comp_stat(ori,stm,dist,lambs)
        s += str(r_e[k][0])+","
        s += str(r_t[k][0])+","
        s += str(r_s[k][0])+","
        s += str(r_d[k][0])+"," 

        
        dist = "em"
        r_st = comp_stat_stb(ori,stb,dist,lambs)
        r_e = comp_stat(ori,etn,dist,lambs)
        r_t = comp_stat(ori,tag,dist,lambs)
        r_d = comp_stat(ori,dym,dist,lambs)
        r_s = comp_stat(ori,stm,dist,lambs)
        s += str(r_e[k][0])+","
        s += str(r_t[k][0])+","
        s += str(r_s[k][0])+","
        s += str(r_d[k][0])+"," 

        dist = "ks"
        r_st = comp_stat_stb(ori,stb,dist,lambs)
        r_e = comp_stat(ori,etn,dist,lambs)
        r_t = comp_stat(ori,tag,dist,lambs)
        r_d = comp_stat(ori,dym,dist,lambs)
        r_s = comp_stat(ori,stm,dist,lambs)
        s += str(r_e[k][0])+","
        s += str(r_t[k][0])+","
        s += str(r_s[k][0])+","
        s += str(r_d[k][0])+"," 

        print(s)
        


la025,0.18106990264995987,0.2376599615209426,0.4363356677657082,0.49955667801775644,3.4256639340838193,5.283180315533095,14.360754954185321,13.731392306191896,4.545,20.52,32.230000000000004,36.997,0.146,0.3833333333333333,0.7629999999999999,0.7729999999999999,
la015,0.13061424161995655,0.23973545146277644,0.33270988451717787,0.39472842615291076,3.476189364150686,6.736364025321502,11.508531470981861,12.270582005630617,3.8480000000000003,27.145,20.094,25.884000000000004,0.12100000000000002,0.4833333333333334,0.6289999999999999,0.7160000000000001,
la001,0.016631256598786538,0.06480078077021854,0.01765966924456175,0.06488015373155796,0.44415139983001006,0.25529366504985906,0.25684776305576723,1.668175898623463,0.15000000000000008,11.094999999999999,0.15400000000000005,0.46900000000000003,0.064,0.47000000000000003,0.08800000000000001,0.253,
_metala025,0.3739407895060446,0.3550151792475732,0.5303816066941316,0.3941640773839743,9.53110055895781,9.207436198853436,16.107462241569152,12.05335077

# RW

In [66]:
def load_cov(file_name,p):
    ori_cov = np.load("dynamic_results/"+file_name+"/Multiple_run/coverage"+p+"/orig.npy")
    sta_cov = np.load("dynamic_results/"+file_name+"/Multiple_run/coverage"+p+"/stab.npy")
    etn_cov = np.load("dynamic_results/"+file_name+"/Multiple_run/coverage"+p+"/etn.npy")
    stm_cov = np.load("dynamic_results/"+file_name+"/Multiple_run/coverage"+p+"/stm.npy")
    tag_cov = np.load("dynamic_results/"+file_name+"/Multiple_run/coverage"+p+"/tag.npy")
    dym_cov = np.load("dynamic_results/"+file_name+"/Multiple_run/coverage"+p+"/dym.npy")
    
    return ori_cov,sta_cov,etn_cov,stm_cov,tag_cov,dym_cov

def load_mfpt(file_name,p):
    ori_mfpt = np.load("dynamic_results/"+file_name+"/Multiple_run/MFPT"+p+"/orig.npy")
    sta_mfpt = np.load("dynamic_results/"+file_name+"/Multiple_run/MFPT"+p+"/stab.npy")
    etn_mfpt = np.load("dynamic_results/"+file_name+"/Multiple_run/MFPT"+p+"/etn.npy",allow_pickle=True)
    stm_mfpt = np.load("dynamic_results/"+file_name+"/Multiple_run/MFPT"+p+"/stm.npy",allow_pickle=True)
    tag_mfpt = np.load("dynamic_results/"+file_name+"/Multiple_run/MFPT"+p+"/tag.npy",allow_pickle=True)
    dym_mfpt = np.load("dynamic_results/"+file_name+"/Multiple_run/MFPT"+p+"/dym.npy",allow_pickle=True)
    
    return ori_mfpt,sta_mfpt,etn_mfpt,stm_mfpt,tag_mfpt,dym_mfpt


def comp_stat_stb(ori,competitor,dist,names):
    
    res = dict()
    c = 0
    for comp in competitor:
        tmp = []
        if dist == "js":
            val = compute_kl_divergence(ori[c], comp, n_bins=50,js=True)
        elif dist == "kl":
            val = compute_kl_divergence(ori[c], comp, n_bins=50,js=False)
        elif dist == "em":
            val = em(ori[c],comp)
        elif dist == "ks":
            val = (ks_2samp(ori[c],comp)[0])

        tmp.append(val)
            
        res[names[c]] = (np.mean(tmp),np.std(tmp))
        c = c + 1 
        
    return res



def compute_cov_mfpt_give_file(file_name,p,fun,dist):

    ori_cov,sta_cov,etn_cov,stm_cov,tag_cov,dym_cov = fun(file_name,p)

    ec_kl = []
    tc_kl = []
    sc_kl = []
    dc_kl = []

    if dist == "ks":
        for i in etn_cov:
            ec_kl.append(ks_2samp(ori_cov,i)[0])
        for i in tag_cov:
            tc_kl.append(ks_2samp(ori_cov,i)[0])
        for i in dym_cov:
            dc_kl.append(ks_2samp(ori_cov,i)[0])
        for i in stm_cov:
            sc_kl.append(ks_2samp(ori_cov,i)[0])
        stb = ks_2samp(ori_cov,sta_cov)[0]
        
    elif dist == "js":
        for i in etn_cov:
            ec_kl.append(compute_kl_divergence(ori_cov,i, n_bins=50,js=True))
        for i in tag_cov:
            tc_kl.append(compute_kl_divergence(ori_cov,i, n_bins=50,js=True))
        for i in dym_cov:
            dc_kl.append(compute_kl_divergence(ori_cov,i, n_bins=50,js=True))
        for i in stm_cov:
            sc_kl.append(compute_kl_divergence(ori_cov,i, n_bins=50,js=True))
            
        stb = compute_kl_divergence(ori_cov,sta_cov, n_bins=50,js=True)
    
    elif dist == "kl":
        for i in etn_cov:
            ec_kl.append(compute_kl_divergence(ori_cov,i, n_bins=50,js=False))
        for i in tag_cov:
            tc_kl.append(compute_kl_divergence(ori_cov,i, n_bins=50,js=False))
        for i in dym_cov:
            dc_kl.append(compute_kl_divergence(ori_cov,i, n_bins=50,js=False))
        for i in stm_cov:
            sc_kl.append(compute_kl_divergence(ori_cov,i, n_bins=50,js=False))
            
        stb = compute_kl_divergence(ori_cov,sta_cov, n_bins=50,js=False)
    
        
    elif dist == "em":
        for i in etn_cov:
            ec_kl.append(em(ori_cov,i))
        for i in tag_cov:
            tc_kl.append(em(ori_cov,i))
        for i in dym_cov:
            dc_kl.append(em(ori_cov,i))
        for i in stm_cov:
            sc_kl.append(em(ori_cov,i))
        stb = em(ori_cov,sta_cov)
    
    else:
        print("error"*100)
    
    

    res = [stb, np.mean(ec_kl),np.mean(tc_kl),
           np.mean(sc_kl),np.mean(dc_kl)]

    return res

[0.7049999999999998,
 10.624100000000002,
 36.62566666666667,
 6.952500000000001,
 15.902899999999999]

In [82]:
file_name = "High_School11"
lambs_all = ["la025","la015","la001"]
picchi = ["","_meta","_2picco"]


for picco in picchi:
    s = picco + " ,"
    stb,etn,tag,dym,stm = compute_cov_mfpt_give_file(file_name,picco,load_cov,"js")
    s += str(etn)+","
    s += str(tag)+","
    s += str(dym)+","
    s += str(stm)+"," 

    stb,etn,tag,dym,stm = compute_cov_mfpt_give_file(file_name,picco,load_cov,"kl")
    s += str(etn)+","
    s += str(tag)+","
    s += str(dym)+","
    s += str(stm)+"," 

    stb,etn,tag,dym,stm = compute_cov_mfpt_give_file(file_name,picco,load_cov,"em")
    s += str(etn)+","
    s += str(tag)+","
    s += str(dym)+","
    s += str(stm)+"," 
    
    stb,etn,tag,dym,stm = compute_cov_mfpt_give_file(file_name,picco,load_cov,"ks")
    s += str(etn)+","
    s += str(tag)+","
    s += str(dym)+","
    s += str(stm)+"," 
    
    print(s)
    

for picco in picchi:
    s = picco + " ,"
    stb,etn,tag,dym,stm = compute_cov_mfpt_give_file(file_name,picco,load_mfpt,"js")
    s += str(etn)+","
    s += str(tag)+","
    s += str(dym)+","
    s += str(stm)+"," 

    stb,etn,tag,dym,stm = compute_cov_mfpt_give_file(file_name,picco,load_mfpt,"kl")
    s += str(etn)+","
    s += str(tag)+","
    s += str(dym)+","
    s += str(stm)+"," 

    stb,etn,tag,dym,stm = compute_cov_mfpt_give_file(file_name,picco,load_mfpt,"em")
    s += str(etn)+","
    s += str(tag)+","
    s += str(dym)+","
    s += str(stm)+"," 
    
    stb,etn,tag,dym,stm = compute_cov_mfpt_give_file(file_name,picco,load_mfpt,"ks")
    s += str(etn)+","
    s += str(tag)+","
    s += str(dym)+","
    s += str(stm)+"," 
    
    print(s)

 ,0.23214387000094655,0.434527202885541,0.13664703515346613,0.39536672045404647,2.4849059218099367,7.3221755840003135,2.2964990082886954,8.71534618505758,10.624100000000002,36.62566666666667,6.952500000000001,15.902899999999999,0.5502,0.8933333333333332,0.3559,0.7175999999999999,
_meta ,0.04177698541720046,0.058730623007760246,0.20300428374924567,0.28475297632186464,0.17778975889732726,0.3159365931622194,3.875254605114532,1.9340074039350927,1.0615,5.174499999999999,1.6025,4.757,0.11,0.259,0.273,0.6185,
_2picco ,0.2629031745740039,0.21822898649270617,0.05395400268822901,0.6438754319299282,2.2773974288846137,0.8300988795481887,1.2957476379984025,15.932071484023737,10.9775,32.185,1.5935000000000001,21.618499999999997,0.6054999999999999,0.7545,0.16049999999999998,0.96,
 ,0.047144402605426405,0.03108927767816576,0.498741162315606,0.3727264701720285,0.24758607485556672,0.12987559779294597,3.826690651734915,4.5837218294820214,41.841245807914184,87.53680017933907,404.12367886946765,279.0993772